In [ ]:
import numpy as np
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt
import librosa

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def create_dataset(folder_path): 
  #This function creates images and species array from the image folder path.
  images = []
  names = []
  max_length = 0
  for file in os.listdir(folder_path):
    image_path = os.path.join(folder_path, file)
    #Resizing images and converting images to grayscale.
    size = Image.open(image_path).size
    image = np.array(Image.open(image_path).resize((int(size[0]/2),int(size[1]/2))))
    image = image.astype('float32')
    image = np.delete(image, [3], axis=2)
    #Normalization
    image /= 255.0
    images.append(image)
    names.append(file[:-4])
    if image.shape[1]>max_length:
      max_length = image.shape[1]

  for i in range(len(images)):
    images[i] = librosa.util.fix_length(images[i], max_length, axis=1)

  images = np.array(images)

  df = pd.read_csv('/content/drive/MyDrive/Deneme/clotho_metadata_development.csv')

  keywords = []
  for i in names:
    keywords.append(df.keywords[df.file_name == i].to_list()[0].lower().split(';'))

  return names,images,keywords

In [ ]:
names, images, keywords = create_dataset("/content/drive/MyDrive/Train Images")

In [ ]:
all_words = []
for i in keywords:
  for j in i:
    if len(j)>2:
      all_words.append(j)

len(all_words)

21309

In [ ]:
from collections import Counter

frequency_list = Counter(all_words).most_common(300)

In [ ]:
keywords_updated = []
for j in range(len(keywords)):
  keyword_list = []
  for i in frequency_list:
    if i[0] in keywords[j]:
      keyword_list.append(i[0])
  keywords_updated.append(keyword_list)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
multihot = mlb.fit_transform(keywords_updated)
class_labels = list(mlb.classes_)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, multihot, test_size=0.20, random_state=33)
print(X_train.shape,X_test.shape)

(2314, 64, 646, 3) (579, 64, 646, 3)


In [ ]:
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

def define_model():

    input_shape=(64,646,3)

    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape= input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))  #Adding the Hidden layer
    model.add(Dense(300, activation='softmax')) #Adding the Output Layer

    
    # compile model
    opt = SGD(learning_rate = 0.0001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model = define_model()

In [ ]:
history = model.fit(X_train, y_train, epochs=10, validation_data= (X_test, y_test))

Epoch 1/10
73/73 [==============================] - 61s 827ms/step - loss: 109.9500 - accuracy: 0.0428 - val_loss: 1058.3391 - val_accuracy: 0.0190
Epoch 2/10
73/73 [==============================] - 58s 797ms/step - loss: 237413760.0000 - accuracy: 0.0320 - val_loss: 53.9832 - val_accuracy: 0.0743
Epoch 3/10
73/73 [==============================] - 57s 786ms/step - loss: 3469.8748 - accuracy: 0.0415 - val_loss: 61656.1328 - val_accuracy: 0.0190
Epoch 4/10
73/73 [==============================] - 57s 777ms/step - loss: 2821117952.0000 - accuracy: 0.0160 - val_loss: 23.5143 - val_accuracy: 0.0052
Epoch 5/10
73/73 [==============================] - 57s 778ms/step - loss: 71.1672 - accuracy: 0.0039 - val_loss: 23.4016 - val_accuracy: 0.0035
Epoch 6/10
73/73 [==============================] - 57s 775ms/step - loss: 24.4315 - accuracy: 0.0052 - val_loss: 23.5192 - val_accuracy: 0.0639
Epoch 7/10
73/73 [==============================] - 57s 775ms/step - loss: 26.2187 - accuracy: 0.0601 - val